In [1]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [3]:
# Importing data
dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

# convert class
num_classes = 4
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model initialization
model = keras.Sequential([keras.Input(shape=(225, 300, 3))])


# Defining building blocks
def add_conv_lyr(num_neurons, k_size, act_func, times=1):
    for i in range(times):
        model.add(layers.Conv2D(num_neurons, kernel_size=k_size, activation=act_func))


def add_max_pool_lyr(p_size, times=1):
    for i in range(times):
        model.add(layers.MaxPooling2D(pool_size=p_size))


def add_dense_lyr(num_neurons, act_func, times=1):
    for i in range(times):
        model.add(layers.Dense(num_neurons, activation=act_func))


# Adding logic to the model
add_conv_lyr(64, (11, 11), 'relu', 1)
add_max_pool_lyr((4, 4))
add_conv_lyr(48, (7, 7), 'relu', 1)
add_max_pool_lyr((3, 3))
add_conv_lyr(32, (3, 3), 'relu', 1)
add_max_pool_lyr((2, 2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
add_dense_lyr(64, 'relu', 3)

# Ending model
add_dense_lyr(num_classes, 'softmax')
model.summary()

batch_size = 34 * 2
epochs = 15

opt = keras.optimizers.Adam(learning_rate=0.0013, beta_1=0.87, beta_2=0.9, epsilon=1.9e-07)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print('\n-------------------- MODEL COMPILED AND READY --------------------\n\n')

Training set shape: (3400, 225, 300, 3)
Testing set shape: (600, 225, 300, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 215, 290, 64)      23296     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 53, 72, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 66, 48)        150576    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 22, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 20, 32)        13856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 10, 32)         0         
____________________________________________

In [7]:
import time
start_time = time.time()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=1.5/8.5)
print((time.time() - start_time) / 60, 'minutes')

Epoch 1/15
42/42 [==============================] - 334s 8s/step - loss: 0.0088 - accuracy: 0.9968 - val_loss: 0.0814 - val_accuracy: 0.9850
Epoch 2/15
42/42 [==============================] - 339s 8s/step - loss: 0.0378 - accuracy: 0.9875 - val_loss: 0.1096 - val_accuracy: 0.9583
Epoch 3/15
42/42 [==============================] - 370s 9s/step - loss: 0.0356 - accuracy: 0.9886 - val_loss: 0.0310 - val_accuracy: 0.9917
Epoch 4/15
42/42 [==============================] - 348s 8s/step - loss: 0.0206 - accuracy: 0.9932 - val_loss: 0.0415 - val_accuracy: 0.9883
Epoch 5/15
42/42 [==============================] - 346s 8s/step - loss: 0.0089 - accuracy: 0.9971 - val_loss: 0.0481 - val_accuracy: 0.9900
Epoch 6/15
42/42 [==============================] - 347s 8s/step - loss: 0.0067 - accuracy: 0.9975 - val_loss: 0.0683 - val_accuracy: 0.9817
Epoch 7/15
42/42 [==============================] - 374s 9s/step - loss: 0.0238 - accuracy: 0.9914 - val_loss: 0.0523 - val_accuracy: 0.9900
Epoch 8/15
42